# SoundDevice: modelos de salida de audio

SoundDevice posibilita varias formas de reproducir audio

- *play*: es el modelo más básico, útil para reproducir muestras enteras (como un *wav*) **sin interacción** con el usuario

- Reproducción por CHUNKs o (pequeños) bloques de señal de tamaño fijo: permite la interacción con el usuario entre bloques. Se hace mediante *streams*. A su vez, puede hacerse de dos modos:

    - En la hebra de ejecución principal, con un bucle que vaya escribiendo los chunks en el stream de salida

    - En una hebra secundaria, mediante un método **callback**

#### El último modelo es el más útil para trabajar con señales en tiempo real, secuenciadores, etc

A continuación presentamos las 3 versiones de manera simplificada, para reproducir la señal de un oscilador

In [4]:
%%writefile consts.py
# común para todos los scripts

import numpy as np
import sounddevice as sd

SRATE = 48000  # frecuencia de muestreo, para todo el proyecto
CHUNK = 1024   # tamaño de los CHUNKs o bloque

Overwriting consts.py


# Versión 1: reproductor básico

SoundDevice proporciona un *play* para reproducción básica: recibe un array con todo el audio

In [5]:
# Para mostrarlo, generamos una pequeña muestra de audio
from consts import *
# array con 2 segundos de audio de un oscilador a 110 Hz
data = np.sin(2*np.pi*np.arange(2*SRATE)*110/SRATE)

In [6]:
# REPRODUCTOR BÁSICO
# lanzamos todo el audio y esperamos a que termine

sd.play(data, SRATE)
sd.wait()

# Versión 2: reproductor por CHUNKs en hebra principal 

Se utiliza un *stream* para ir enviando la señal por CHUNKs (bloques), pero aún se ejecuta en hebra principal

In [7]:
# REPRODUCTOR POR CHUNKS "semi bloqueante"
# lanzamos el audio por bloques de tamaño CHUNK en la hebra principal de ejecucion
# -> la ejecución se bloquea hasta que se termina de reproducir el bloque

from consts import *

# stream de salida
stream = sd.OutputStream( # creamos stream 
    samplerate = SRATE,            # frec de muestreo 
    blocksize  = CHUNK,            # tamaño del bloque
    channels   = len(data.shape))  # num de canales
stream.start() # arrancamos stream

numBloque = 0  # contador de bloques
end = False    # flag de fin de audio
while not(end): 
    print(f"Procesando bloque: {numBloque}")
    bloque = data[numBloque*CHUNK : (numBloque+1)*CHUNK]
    if bloque.size==0: break
    stream.write(np.float32(bloque)) # escribimos al stream de sounddevice
    numBloque += 1

stream.stop() # paramos stream
stream.close() # cerramos stream    

Procesando bloque: 0
Procesando bloque: 1
Procesando bloque: 2
Procesando bloque: 3
Procesando bloque: 4
Procesando bloque: 5
Procesando bloque: 6
Procesando bloque: 7
Procesando bloque: 8
Procesando bloque: 9
Procesando bloque: 10
Procesando bloque: 11
Procesando bloque: 12
Procesando bloque: 13
Procesando bloque: 14
Procesando bloque: 15
Procesando bloque: 16
Procesando bloque: 17
Procesando bloque: 18
Procesando bloque: 19
Procesando bloque: 20
Procesando bloque: 21
Procesando bloque: 22
Procesando bloque: 23
Procesando bloque: 24
Procesando bloque: 25
Procesando bloque: 26
Procesando bloque: 27
Procesando bloque: 28
Procesando bloque: 29
Procesando bloque: 30
Procesando bloque: 31
Procesando bloque: 32
Procesando bloque: 33
Procesando bloque: 34
Procesando bloque: 35
Procesando bloque: 36
Procesando bloque: 37
Procesando bloque: 38
Procesando bloque: 39
Procesando bloque: 40
Procesando bloque: 41
Procesando bloque: 42
Procesando bloque: 43
Procesando bloque: 44
Procesando bloque: 4

# Versión 3: reproducción con callBack (no bloqueante) y enrutado de señal (objeto con método next)

Tendremos un **hebra de ejecución** independiente, donde estará con soundDevice procesando señal:

- Esta hebra procesa el audio que le llegue a través de (la variable) **input**

- **input** es *cualquier* generador de señal, i.e., cualquier objeto con un **método next** que genere un CHUNK de audio con cada llamada al mismo: es un iterador que va devolviendo CHUNKs de señal en cada sucesiva llamada

Para el siguiente ejemplo, utilizaremos un oscilador simple como generador de señal: clase Osc

In [1]:
%%writefile osc.py
from consts import *

class Osc:
    def __init__(self,freq=440.0,amp=1.0,phase=0.0):
        self.freq = freq
        self.amp = amp
        self.phase = phase
        self.frame = 0 # frame inicial

    def next(self):            
        out = self.amp*np.sin(2*np.pi*(np.arange(self.frame,self.frame+CHUNK))*self.freq/SRATE+self.phase)        
        self.frame += CHUNK # actualizamos el frame
        return out

Overwriting osc.py


Definimos un stream de sounDevice, pero ahora con callBack.

- En el callback simplemente pedimos un CHUNK a la señal *input* y lo copiamos en *outdata*



In [13]:
from consts import *
from osc import *

input = None
def callback(outdata, frames, time, status):    
    global input
    #print('entro')
    if input is not None: 
        bloque = input.next()    
        outdata[:] = bloque.reshape(-1,1) # convertimos formato (CHUNK,) a (CHUNK,1) para que adecuarlo a sounddevice
    else:
        outdata[:] = np.zeros((CHUNK,1)) # si no hay datos, reproducimos silencio
            
# stream de salida con callBack
stream = sd.OutputStream(samplerate=SRATE, channels=len(data.shape), callback=callback, blocksize=CHUNK)
stream.start()


In [8]:
# Conectamos el oscilador al input stream de salida
input = Osc(110,1.0,0.0) # oscilador a 110 Hz
# empieza a sonar!!

# y no para: el callback va demandando señal al oscilador que la va produciendo en tiempo real


In [9]:
# para pararlo, simplemente ponemos input a None (desconectamos el oscilador)
input = None # paramos el oscilador

In [ ]:
# Ahora, conectamos el oscilador
input = Osc(110,1.0,0.0) # oscilador a 110 Hz
# empieza a sonar!

 # esperamos 2 segundos
sd.sleep(2000)

# desconectamos el oscilador (se para)
input = None # paramos el oscilador


In [ ]:
# al final del todo podemos paramos stream
stream.stop()
stream.close()


In [ ]:
print(input)

input = None

# TkInter para entrada/salida no bloquente... y GUIs!

Cedemos el control de la hebra ppal a TkInter

- Y en particular cedemos el control de ejecución: toda la interacción con el sistema será a través de ventanas

In [14]:
from consts import *
from tkinter import *
from osc import *

root=Tk()

# Caja de texto
text = Text(root,height=6,width=60)
text.pack(side=BOTTOM)
text.insert(INSERT,"Press 'a' to activate note and 'space' to deactivate\n")

# call back para la pulsación de teclas
def key_down(event):
    global input  # conexión con sounddevice
    if event.char=='a': 
        print('note activated')
        input = Osc(110)  # enrutamos la señal de la nota al "input" del stream
    elif event.char==' ': 
        print('note deactivated')
        input = None

# enlace de la pulsación de teclas con la función key_down
text.bind('<Key>', key_down)


# arrancamos la ventana: la ejecución queda bloqueada hasta que se cierre la ventana
root.mainloop()

# limpieza..
#stream.stop()
#stream.close()

note activated
note deactivated
